# Instance Weighting for Neural Machine Translation Domain Adaption
Rui Wang, Masao Utiyama


## 0. あぶすと
instance weightingはPBMTにおけるドメイン適合(転移学習)で広く用いられている手法である.  
しかしながら、NMTは線形モデルでないため、これをNMTに直接適用することは困難である.  
本論文では、NMTのドメイン適合において、2つのinstance weighting(文重みとドメイン重み)の動的な重みを学習する戦略を提案する.  
提案手法はIWSLTの英独/英仏タスクにおいて、NMTの性能をBLEU値で2.7-6.7向上させ、既存のベースラインよりも1.6-3.6上昇させる結果を残した.  

## 1. いんとろ
SMTでは、ドメイン外コーパスと呼ばれる無関係な追加コーパスは、TEDトークやIWSLTタスクのような一部のドメインやタスクに貢献しないことが知られている.  
特定のドメインや言語における不十分なデータ欠如の問題を克服するため、様々なPBSMTにおけるドメイン適合手法が提案されている.  

1. データ選択: ドメイン内とドメイン外のデータからそれぞれ訓練されたモデルを使用し、ドメイン外のデータにスコアを与える  
1. モデル線形補完:  各コーパスに個別に対応する複数のPBSMTモデル(翻訳モデル、言語モデル、並び替えモデル)を訓練する。次に、これらのモデルを組み合わせて性能を向上させる.  
1. Instance Weighting: いくつかのNLPドメイン適合タスク(品詞タギング、固有タイプ分類、特にPBNMT)で用いられる手法。はじめに各インスタンス/ドメインに対しルールベース、統計的手法を用いてスコア(重み)をつけ、それから与えられたインスタンス/ドメインの重みを用いてPBNMTモデルを訓練する.  

NMTのドメイン適合では、文選択がよく用いられる.  
一方で、NMTは線形モデルでないためにモデル線形補間はNMTに直接適用されにくい.  
NMTのモデル組み合わせには2つの方法がある.  

i ) ドメイン内モデルとドメイン外モデルをアンサンブルに用いる  
ii ) NMTを更に訓練する(fine-tuning: 他のデータセットで学習済のモデルのパラメータをコピーし、新しいモデルの初期値として扱う)手法  
この学習では2つのステップからなる。はじめに、NMTシステムをドメイン外データを用いて学習し、それからドメイン内のデータを用いて学習する.  

近年では、Chuらが単語レベルでのドメイン特徴を単語埋め込み層に適用した場合のNMTのさらなる訓練とドメイン操作の比較を行った.  
この手法では、比較のためのベースラインを提供した.  

筆者の知る限りにおいて、NMTにおけるインスタンスの重み付けに関する既存手法は存在しない.  
主な課題として, NMTがインスタンス重みを直接統合できる線形モデルまたは線形モデルの組み合わせではないことが挙げられる.  

この問題を克服するため、インスタンス重みをNMT目的関数に統合することを試みる.  
NMTにインスタンス重み付けを適用するため、文重み付けとドメイン重み付けの2つの手法を提案する.  
さらに、提案されたドメイン重み付けを調整する動的重み学習戦略も提案する.  



## 2. NMTの背景
あてんちょんべーすどNMTはソース文xからターゲット文yを翻訳する条件付き確率p(y|x)を直接モデリングする手法であり, p(y|x)は以下のごちゃごちゃした式から求まる  
ここで、gは変換関数であり、語彙サイズのベクトルを出力し、sjはRNNの隠れユニットであり、cjはソース注釈Hxの重み付き総和である.  
NMTの訓練目標は以下のように定式化される  

$$ J = \sum_{(x,y)\in D}log p(y|x) $$

## 3. NMTのためのインスタンス重み
本紙では、インスタンス重みをNMTの目的関数に導入する.  
主な仮説は、ドメイン内のデータはドメイン外のデータよりもNMTの目的関数においてより大きい重み付けがなされるべきということである.  

訓練コーパスDをドメイン内コーパス$D_{in}$とドメイン外コーパス$D_{out}$に分割する.  
そして、(2)式を以下のように書き直す.  

$$ J = ( \sum_{<x,y>\in \mathbb{D}_{in}} log p(y|x) + \sum_{<x',y'>\in D_{out}} log p(y'|x') ) $$

ここで、< x,y \>は多言語の文対である.  

## 3.1 文重み
一般的な手法は、各文に重みを与えることである.  
Axelrodらは、ドメイン外データにはドメイン内データに近しい疑似ドメイン内データが存在することを述べた.  
提案手法では、二ヶ国語間にクロスエントロピー法を適用し, 各\< x,y \>に重みλiとしてスコア付けを行う. これは高いほうがよい.  

$$ \lambda_i = \delta(H_{out}(x_i) - H_{in}(x_i) + H_{out}(y_i) - H_{in}(y_i) ) $$

例として$H_{in}(x_i)$を考える. これは文xiとドメイン内言語モデルのクロスエントロピーを示す. Min-max正規化δを用い, λiは[0,1]の範囲に正規化される.  

$$ \delta(\lambda_i) = \frac{\lambda_i-\lambda_{min}}{\lambda_{max}-\lambda_{min}} $$

ドメイン内データにおけるλは直接1に設定される. 文重みJswにより更新された目的関数は次のように書き直せる.  

$$ J_{sw} = \sum_{<x_i,y_i \in \mathbb{D}>} \lambda_i log p(y_i|x_i) $$

## 3.2 ドメイン重み
別の方法として、目的関数における各ドメインの重みを調節することが挙げられる. 提案手法では, ドメイン内データの重みパラメータ$\lambda_in$を設計することを考える. 　
ドメイン重みJdwにより更新された目的関数は, 次のように推定できる.  

$$ J_{sw} = \lambda_{in}\sum_{ (x_i,y_i) \in D_{in}} log p(y|x) + \sum_{(x',y')\in D_{out}} log p(y'|x')$$

### 3.2.1 バッチ重み
単純なドメイン重みの実装は、各NMTのミニバッチにおけるドメイン内データとドメイン外データの割合を調整することで実現できる.  
つまり、ミニバッチに含まれるドメイン内文の数を増やすことでドメイン内重みを向上することができる.  
各NMTミニバッチにおけるドメイン内データの割合Rinは次のように計算できる  

$$ R_{in} = \frac{|\hat{ D }_{in}|}{|\hat{D}'_{in}| + |\hat{D}'_{out}|} = \frac{\lambda_{in}}{\lambda_{in} + 1} $$

ここで$|\hat{D}_{in}|$と|$\hat{D}_{out}$|は各ミニバッチにおけるドメイン内/ドメイン外データの文数をそれぞれ表す.  

IWSLTの英独コーパスを例に取ると、データ全体に対するドメイン内データの元の割合$R_{in}$はおよそ1:20である.  
これは、サイズ80のミニバッチでは、およそ4つのドメイン内データからの文が含まれ、76がドメイン外のデータである.  
バッチ重みでは、Rinを1:2に手動でセットすることができる.  
この場合、40のドメイン内データと40のドメイン外データを各ミニバッチにロードすることが可能となる  

実際には、2つのデータイテレータを作成し、1つはドメイン内データ、もう1つはドメイン外データに対応する.  
ドメイン内データとドメイン外データの両方がランダムにシャッフルされ、各エポック(が始まる？)前に対応するデータイテレータにロードされる.  
各ミニバッチにおいて、これら2つのデータイテレータから取り出されるデータは割合Rinによって定義される.  
ドメイン外データの数はドメイン内データの数よりもはるかに大きいため、データが複数のエポックで読み込まれ、訓練される一方で、ドメイン外のデータは同時に1つのエポックでしか訓練されない.  

### 3.2.2 動的重み付け
バッチ重み付け調整では、1つの方法として複数のシステムの重みを固定し、開発データで最も良い結果を残したシステムを選ぶ.  
これに対し、提案手法ではバッチ重み付けを動的に学習することを試みた.  
10回の評価でトレーニングコストが減少しない場合、ミニバッチ内におけるドメイン内データの割合を10%増加させた(トレーニングコストは1Kバッチトレーニングごとの開発データで評価される)  



## 4 実験
### 4.1 データ・セット
提案手法を評価するため、WMTコーパスをIWSLT(主にTEDトークの内容を含む)コーパスに適用する.  
IWSLT2015コーパスのデータをドメイン内訓練データとして使用し、WMT2014コーパスをドメイン外データとして用いる.  
(英独、英仏タスクを用意)  

### 4.2 NMTシステム
提案手法をNematusに実装した。このシステムはsotaなフレームワークの一種である.  
すべてのNMTシステムにはNematus標準の設定を適用した:  
単語埋め込み次元数 - 620  
隠れ層数 - 1000  
バッチサイズ - 80  
最大系列長 - 50  
デコード時のビーム長 - 10  
語彙数 - 30K(ドメイン内/ドメイン外どちらも)  
ドロップアウト - 標準(?)  

各NMTはADADELTAを用い500Kバッチに渡り訓練される.  
とにかく計算量が多い  

### 4.3 結果と考察
IWSLTの英独、英仏についてBLEU値を計測した  
比較対象として、インドメイン、アウトドメインとその両方をデータセットに含むSMT、NMTの他、インドメインとアウトドメインそれぞれでモデルを構築し、デコード時にアンサンブルして用いる"ensemble"NMTと手動で訓練データのイン/アウトの比率を1:1にした"sampler"NMTを用いた  
またAxelrodらとKobusらのドメイン操作モデルも比較対象として用いた  
翻訳性能の指標にはpaired bootstrap re-sampling test(?)のBLEU値を採用した  
単純にアウトドメインをインドメインにつっこんだだけやアウトドメインをそのまま用いた場合、インドメインだけを使った元のNMT(in-NMT)よりも性能が下がった   
提案したインスタンス重み付き手法はin-NMTよりも2.7-6.7ポイントほど上昇し、既存ベースラインのうち最良のものよりも1.6-3.6ポイント上昇した  
提案手法の中で最もよい性能を示したのはバッチ重みという最も単純なモデルであった  
理由としては、  
a) バッチ重み付け手法は各ミニバッチにおいて直接インドメインのレートをバランス取れるからインドメインデータがスパースな問題が解決できる  
b) バッチ重みはでぶろっぷデータで調整できるので、NMTトレーニングの前に学習し修正される文重み付け手法とは違う  
動的重み調整戦略は固定重み調整より0.6-2.4ポイント上昇した  



# 5 ですかっしょん
## 5.1 重み調整
めんどくさ  
